In [69]:
import pandas as pd
from pathlib import Path
import pyarrow as pa  # not yet needed, might need it later
import pyarrow.parquet as pq
import pyarrow.compute as pc  # not yet needed, might need it later.

## Reading Parquet-systems metrics metadata
First, look at the parquet metrics and see which ones you wish to study.

In [70]:
metrics_dir = Path('../../data/raw/parquet-metrics/')
# Note: We generally give a directory, rather than an individual file, to the next step.
metrics_pq = pq.ParquetDataset(metrics_dir)
metrics_df = metrics_pq.read().to_pandas()

In [71]:
metrics_df.head()

,system_id,metric_id,sensor_name,common_name,raw_units,units,calc_scale,calc_offset,calc_details,aggregation_type,source_type,source_id,comments,standard_name
0,10,422,dc_power,DC power,W,W,1.0,0.0,,avg,None,NaN,,dc_power__422
1,10,423,ac_power,AC power,W,W,1.0,0.0,,avg,None,NaN,,ac_power__423
2,10,427,ac_current,AC current,A,A,1.0,0.0,,avg,None,NaN,,ac_current__427
3,10,426,ac_voltage,AC voltage,V,V,1.0,0.0,,avg,None,NaN,,ac_voltage__426
4,10,425,dc_pos_current,DC current,A,A,1.0,0.0,,avg,None,NaN,,dc_pos_current__425


In [72]:
metrics_df.columns

Index(['system_id', 'metric_id', 'sensor_name', 'common_name', 'raw_units',
       'units', 'calc_scale', 'calc_offset', 'calc_details',
       'aggregation_type', 'source_type', 'source_id', 'comments',
       'standard_name'],
      dtype='object')

Let's pick a system -- say, '2' , and restrict our attention to that system

In [73]:
system_id = 2
metrics_our_system = metrics_df.loc[metrics_df.loc[:,'system_id'] == system_id]

In [74]:
metrics_our_system

,system_id,metric_id,sensor_name,common_name,raw_units,units,calc_scale,calc_offset,calc_details,aggregation_type,source_type,source_id,comments,standard_name
1297,2,346,dc_power,DC power,W,W,1.0,0.0,,avg,None,NaN,,dc_power__346
1298,2,348,dc_pos_current,DC current,A,A,1.0,0.0,,avg,None,NaN,,dc_pos_current__348
1299,2,351,das_battery_voltage,DC voltage battery,V,V,1.0,0.0,,avg,None,NaN,,das_battery_voltage__351
1300,2,345,poa_irradiance,Irradiance POA,W/m^2,W/m^2,1.0,0.0,,avg,None,NaN,,poa_irradiance__345
1301,2,349,module_temp_1,Temperature module,C,C,1.0,0.0,,avg,None,NaN,,module_temp_1__349
1302,2,347,dc_pos_voltage,DC voltage,V,V,1.0,0.0,,avg,None,NaN,,dc_pos_voltage__347
1303,2,350,das_temp,Temperature panel,C,C,1.0,0.0,,avg,None,NaN,,das_temp__350


For the current purposes, let's grab irradiance and temperature.  
*However*, data is stored in the main parquet framework by metric_id,
so just manually choose the ids you want.

In [75]:
my_metric_ids = [345, 349, 350]
metrics_our_system_pruned = metrics_our_system.loc[
    metrics_our_system.loc[:, 'metric_id'].isin(my_metric_ids)
]
metrics_our_system_pruned.head()

,system_id,metric_id,sensor_name,common_name,raw_units,units,calc_scale,calc_offset,calc_details,aggregation_type,source_type,source_id,comments,standard_name
1300,2,345,poa_irradiance,Irradiance POA,W/m^2,W/m^2,1.0,0.0,,avg,None,NaN,,poa_irradiance__345
1301,2,349,module_temp_1,Temperature module,C,C,1.0,0.0,,avg,None,NaN,,module_temp_1__349
1303,2,350,das_temp,Temperature panel,C,C,1.0,0.0,,avg,None,NaN,,das_temp__350


Grab the numbers and common names for later.

In [76]:
my_metric_ids = tuple(metrics_our_system_pruned['metric_id'])
my_metric_names = tuple(metrics_our_system_pruned['common_name'])

## Now read the relevant data

We now read the actual dataset, using our metric_ids to filter.

In [77]:
access_system_dir = Path(f'../../data/raw/systems/parquet/{system_id}/')
current_pq = pq.ParquetDataset(access_system_dir,
                               filters= [
                                   ('metric_id', 'in', my_metric_ids)
                               ])


In [78]:
current_df = current_pq.read().to_pandas()

In [79]:
current_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5948604 entries, 0 to 5948603
Data columns (total 4 columns):
 #   Column           Dtype         
---  ------           -----         
 0   measured_on      datetime64[ns]
 1   utc_measured_on  datetime64[ns]
 2   metric_id        int32         
 3   value            float64       
dtypes: datetime64[ns](2), float64(1), int32(1)
memory usage: 158.8 MB


In [80]:
current_df.head()

,measured_on,utc_measured_on,metric_id,value
0,2010-01-21 11:05:00,NaT,345,336.800
1,2010-01-21 11:05:00,NaT,349,18.140
2,2010-01-21 11:05:00,NaT,350,5.610
3,2010-01-21 11:08:00,NaT,350,5.657
4,2010-01-21 11:08:00,NaT,349,17.560


Looks good, but there is a trap hidden in the data.  We should have 3 copies of each measured_on, yes?

In [81]:
current_df.measured_on.value_counts()

measured_on
2020-01-13 15:20:00    6
2020-01-13 15:30:00    6
2020-01-13 14:30:00    6
2020-01-13 15:35:00    6
2020-01-13 16:15:00    6
                      ..
2010-01-21 11:48:00    3
2010-01-21 11:49:00    3
2010-01-21 11:50:00    3
2010-01-21 11:52:00    3
2010-01-21 11:19:00    3
Name: count, Length: 1910170, dtype: int64

Duplicates or something more?  Let's look at one case.

In [82]:
current_df_2020_01_13_320pm = current_df.loc[
    (current_df.loc[:, 'measured_on'].dt.year == 2020)
    & (current_df.loc[:, 'measured_on'].dt.month == 1)
    & (current_df.loc[:, 'measured_on'].dt.day == 13)
    & (current_df.loc[:, 'measured_on'].dt.hour == 15)
    & (current_df.loc[:, 'measured_on'].dt.minute == 20)
]

In [83]:
current_df_2020_01_13_320pm

,measured_on,utc_measured_on,metric_id,value
5948381,2020-01-13 15:20:00,2020-01-13 22:20:00,350,9.479595
5948443,2020-01-13 15:20:00,2020-01-13 22:20:00,345,69.584430
5948444,2020-01-13 15:20:00,2020-01-13 22:20:00,350,9.479595
5948445,2020-01-13 15:20:00,2020-01-13 22:20:00,349,20.423150
5948577,2020-01-13 15:20:00,2020-01-13 22:20:00,345,69.584430
5948578,2020-01-13 15:20:00,2020-01-13 22:20:00,349,20.423150


Phew, it appears to be duplicates!

In [84]:
current_df = current_df.drop_duplicates()

In [85]:
current_df.measured_on.value_counts()

measured_on
2020-01-13 16:45:00    3
2010-01-21 11:05:00    3
2010-01-21 11:08:00    3
2010-01-21 11:09:00    3
2010-01-21 11:10:00    3
                      ..
2010-01-21 11:33:00    3
2010-01-21 11:32:00    3
2010-01-21 11:30:00    3
2010-01-21 11:29:00    3
2010-01-21 11:27:00    3
Name: count, Length: 1910170, dtype: int64

Note that we have all metrics in the same value column.
This is often good for plotting, but not so good for reading.
We make a wide variant

In [86]:
current_df_wide = current_df.pivot(
    index = 'measured_on', 
    columns = 'metric_id',
    values = 'value'
)

In [87]:
current_df_wide.head()

metric_id,345,349,350
measured_on,,,
2010-01-21 11:02:00,332.0,18.63,5.564
2010-01-21 11:03:00,343.1,18.37,5.573
2010-01-21 11:04:00,345.4,18.12,5.591
2010-01-21 11:05:00,336.8,18.14,5.610
2010-01-21 11:06:00,321.9,17.95,5.625


Rename the columns!

In [88]:
changeup_dict = {
    my_metric_ids[j]: my_metric_names[j] for j in range(len(my_metric_ids))
}

In [89]:
current_df_wide = current_df_wide.rename(
    columns=changeup_dict
)

In [90]:
current_df_wide

metric_id,Irradiance POA,Temperature module,Temperature panel
measured_on,,,
2010-01-21 11:02:00,332.00000,18.630000,5.564000
2010-01-21 11:03:00,343.10000,18.370000,5.573000
2010-01-21 11:04:00,345.40000,18.120000,5.591000
2010-01-21 11:05:00,336.80000,18.140000,5.610000
2010-01-21 11:06:00,321.90000,17.950000,5.625000
...,...,...,...
2020-01-13 16:25:00,20.65313,2.181308,9.044684
2020-01-13 16:30:00,17.63354,1.808504,8.955369
2020-01-13 16:35:00,15.80738,1.703745,8.866957


Quick missingness check

In [91]:
missing_count_original = current_df.shape[0] - current_df.count()
missing_count_original

measured_on              0
utc_measured_on    5512416
metric_id                0
value                    0
dtype: int64

In [92]:
missing_count_wide = current_df_wide.shape[0] - current_df_wide.count()
missing_count_wide

metric_id
Irradiance POA        0
Temperature module    0
Temperature panel     0
dtype: int64

Wow, we're lucky!

## Saving the Dataframe

Let's practice saving one of the pruned DataFrames.  pd has a to_parquet_column, thankfully.

Final cleaned data should go in `data/cleaned/systems/parquet/system_id/`,
but for practice, I have `data/parquet_practice` set aside.
You should *not* Git-push the contents of the folder!

The example re-saves the non-wide-data, and will partition it by data-type, not by day.

In [ ]:
test_save_dir = Path('../../data/parquet_practice/system_2')
if not test_save_dir.is_dir():
    test_save_dir.mkdir()
current_df.to_parquet(
    path = test_save_dir,
    partition_cols=['metric_id'],
    index=False
)
# Note: if we saved the wide data, we would probably want to retain the index, as that has the measured dates.

## Retrieve the data we just stored.

In [94]:
retry_pq = pq.ParquetDataset(test_save_dir)
retry_df = retry_pq.read().to_pandas()
retry_df.head()

,measured_on,utc_measured_on,value,metric_id
0,2010-01-21 11:05:00,NaT,336.8,345
1,2010-01-21 11:09:00,NaT,338.9,345
2,2010-01-21 11:11:00,NaT,332.4,345
3,2010-01-21 11:13:00,NaT,345.7,345
4,2010-01-21 11:19:00,NaT,320.8,345


It worked!  And each metric_id is together for further analysis!